In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

# طباعة كل الملفات في مجلد Colab
print(os.listdir('/content'))

Mounted at /content/drive
['.config', 'drive', 'sample_data']


In [2]:
import pandas as pd

# 2. قراءة ملف CSV
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/training data project/data for training.csv')


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

# 1. تحميل البيانات
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/training data project/data for training.csv')
df.columns = df.columns.str.strip()

# 2. فلترة صارمة للبيانات
stopwords = [
    "إلى", "من", "عن", "على", "في", "ب", "ك", "ل", "حتى",
    "ثم", "إن", "أن", "لكن", "لا", "لم", "لن", "ما", "هل", "لما",
    "إذا", "كلما", "أو", "بل", "و", "ف", "نحو", "مثلاً", "كذا", "مع", "عند", "قبل", "بعد", "أمام", "خلف"
]
df = df[df['word'].str.len() > 2]
df = df[~df['word'].isin(stopwords)]

# 3. تنظيف التصنيفات وتحديد الهدف
df['label'] = df['label'].str.strip().str.lower()
df['label'] = df['label'].replace({
    'sound masculine': 'sound_masculine',
    'sound feminine': 'sound_feminine'
})
X = df['word']
y = df['label']
print(df['label'].value_counts())

# 4. التقسيم
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. تحويل النصوص إلى تمثيل TF-IDF
vectorizer = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(2, 6),
    min_df=2
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 6. إعداد نموذج Random Forest
model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)

# 7. تقييم باستخدام F1
X_tfidf_full = vectorizer.transform(X)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X_tfidf_full, y, cv=skf, scoring='f1_macro')
print("📊 نتائج Cross-Validation لـ Random Forest:")
print("F1 Macro Scores:", scores)
print("✅ متوسط F1 Macro:", round(scores.mean(), 4))

# 8. تدريب نهائي
model.fit(X_train_tfidf, y_train)

# 9. اختبار النموذج
y_pred = model.predict(X_test_tfidf)
acc = accuracy_score(y_test, y_pred)

# 10. عرض النتائج
print("📌 Random Forest Classifier")
print("Accuracy:", round(acc * 100, 2), "%")
print("\n🔍 Classification Report:")
print(classification_report(y_test, y_pred))
print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# 11. حفظ النموذج
joblib.dump(model, "rf_final_model.pkl")
joblib.dump(vectorizer, "rf_final_vectorizer.pkl")
print("📦 تم حفظ النموذج والمحول بنجاح")


label
broken             1046
sound_feminine      335
sound_masculine     320
Name: count, dtype: int64
📊 نتائج Cross-Validation لـ Random Forest:
F1 Macro Scores: [0.99662048 0.98995677 0.98326129 0.98667838 0.97672963]
✅ متوسط F1 Macro: 0.9866
📌 Random Forest Classifier
Accuracy: 99.41 %

🔍 Classification Report:
                 precision    recall  f1-score   support

         broken       1.00      0.99      1.00       210
 sound_feminine       1.00      1.00      1.00        67
sound_masculine       0.97      1.00      0.98        64

       accuracy                           0.99       341
      macro avg       0.99      1.00      0.99       341
   weighted avg       0.99      0.99      0.99       341


📊 Confusion Matrix:
[[208   0   2]
 [  0  67   0]
 [  0   0  64]]
📦 تم حفظ النموذج والمحول بنجاح


In [4]:
import joblib
import re

# تحميل النموذج والمحول
model = joblib.load("rf_final_model.pkl")
vectorizer = joblib.load("rf_final_vectorizer.pkl")

# دالة تنظيف الكلمة (إزالة التشكيل وتوحيد الهمزات)
def clean_word(text):
    text = re.sub(r'[\u064B-\u0652]', '', text)  # إزالة التشكيل
    text = re.sub(r'[أإآ]', 'ا', text)           # توحيد الهمزات
    return text.strip()

# ✅ الكلمات التي تريد تجربتها
words = ["مهندسات", "مصممون", "كتب", "معلمات", "طلاب", "أساتذة", "صورة", "ممرضون"]

# تنظيف الكلمات
cleaned_words = [clean_word(w) for w in words]

# تحويل الكلمات إلى خصائص (features)
X_test = vectorizer.transform(cleaned_words)

# التنبؤ
preds = model.predict(X_test)
probs = model.predict_proba(X_test)

# عرض النتائج
for i, word in enumerate(words):
    label = preds[i]
    confidence = probs[i].max() * 100
    print(f"🔠 الكلمة: '{word}' → التصنيف: {label} (الثقة: {confidence:.2f}%)")



🔠 الكلمة: 'مهندسات' → التصنيف: sound_feminine (الثقة: 100.00%)
🔠 الكلمة: 'مصممون' → التصنيف: sound_masculine (الثقة: 98.00%)
🔠 الكلمة: 'كتب' → التصنيف: broken (الثقة: 100.00%)
🔠 الكلمة: 'معلمات' → التصنيف: sound_feminine (الثقة: 99.00%)
🔠 الكلمة: 'طلاب' → التصنيف: broken (الثقة: 98.00%)
🔠 الكلمة: 'أساتذة' → التصنيف: broken (الثقة: 80.00%)
🔠 الكلمة: 'صورة' → التصنيف: broken (الثقة: 98.00%)
🔠 الكلمة: 'ممرضون' → التصنيف: sound_masculine (الثقة: 97.00%)


In [5]:
from google.colab import files

# تحميل الملفين
files.download('/content/rf_final_model.pkl')
files.download('/content/rf_final_vectorizer.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
import joblib
import numpy as np

# تحميل النموذج والمحول
model = joblib.load("rf_final_model.pkl")  # أو أي اسم استخدمته
vectorizer = joblib.load("rf_final_vectorizer.pkl")

# قائمة الكلمات للاختبار
test_words = [
    "مهندسون", "مدارس", "أقلام", "سائقون", "صلوات", "طلاب", "مبدعات", "عناوين",
    "خبراء", "لغويون", "أنظمة", "مناهج", "فتيان", "بيوت", "أفكار", "مديرات",
    "سحرة", "رواد", "علماء", "نساء", "قضاة", "نقباء", "جملاء", "قارئون",
    "فائزات", "مرشدات", "رُسُل", "أُسَر", "أطباء", "رؤساء"
]

# تصنيف كل كلمة
for word in test_words:
    vec = vectorizer.transform([word])
    prediction = model.predict(vec)[0]
    probs = model.predict_proba(vec)[0]
    confidence = np.max(probs) * 100
    print(f"🔠 الكلمة: '{word}' → التصنيف: {prediction} (الثقة: {confidence:.2f}%)")


🔠 الكلمة: 'مهندسون' → التصنيف: sound_masculine (الثقة: 95.00%)
🔠 الكلمة: 'مدارس' → التصنيف: broken (الثقة: 92.00%)
🔠 الكلمة: 'أقلام' → التصنيف: broken (الثقة: 98.00%)
🔠 الكلمة: 'سائقون' → التصنيف: sound_masculine (الثقة: 94.00%)
🔠 الكلمة: 'صلوات' → التصنيف: sound_feminine (الثقة: 64.00%)
🔠 الكلمة: 'طلاب' → التصنيف: broken (الثقة: 98.00%)
🔠 الكلمة: 'مبدعات' → التصنيف: sound_feminine (الثقة: 98.00%)
🔠 الكلمة: 'عناوين' → التصنيف: broken (الثقة: 62.00%)
🔠 الكلمة: 'خبراء' → التصنيف: broken (الثقة: 97.00%)
🔠 الكلمة: 'لغويون' → التصنيف: sound_masculine (الثقة: 82.00%)
🔠 الكلمة: 'أنظمة' → التصنيف: broken (الثقة: 99.00%)
🔠 الكلمة: 'مناهج' → التصنيف: broken (الثقة: 97.00%)
🔠 الكلمة: 'فتيان' → التصنيف: broken (الثقة: 99.00%)
🔠 الكلمة: 'بيوت' → التصنيف: broken (الثقة: 93.00%)
🔠 الكلمة: 'أفكار' → التصنيف: broken (الثقة: 100.00%)
🔠 الكلمة: 'مديرات' → التصنيف: sound_feminine (الثقة: 100.00%)
🔠 الكلمة: 'سحرة' → التصنيف: broken (الثقة: 98.00%)
🔠 الكلمة: 'رواد' → التصنيف: broken (الثقة: 98.00%)
🔠 الكلمة